https://pyomo.readthedocs.io/en/stable/ <br>
https://pyomo.readthedocs.io/en/stable/working_models.html

In [1]:
import pyomo.environ as pyo
import pandas as pd

## Eingabedaten

In [2]:
input_data = pd.read_excel("Inputdata.xlsx", index_col="Land")

input_data

input_data.to_dict()

input_data["H2 Herstellungspreis"].to_dict()

## Solver

In [3]:
solver = pyo.SolverFactory('gurobi') 

## Model

In [4]:
model = pyo.ConcreteModel()

## Länder als itterable

input_data.index

In [5]:
model.country = input_data.index

## Parameter

In [6]:
# Parameter in Model importieren
model.h2_price =  pyo.Param(model.country, initialize=input_data["H2 Herstellungspreis"].to_dict())
model.h2_pipeline =  pyo.Param(model.country, initialize=input_data["H2 Pipeline Preis"].to_dict())
model.h2_ship =  pyo.Param(model.country, initialize=input_data["H2 Schiff Preis"].to_dict())
model.h2_limit = pyo.Param(model.country, initialize=input_data["H2 Export Limit"].to_dict())

model.nh3_price =  pyo.Param(model.country, initialize=input_data["NH3 Herstellungspreis"].to_dict())
model.nh3_pipeline =  pyo.Param(model.country, initialize=input_data["NH3 Pipeline Preis"].to_dict())
model.nh3_ship =  pyo.Param(model.country, initialize=input_data["NH3 Schiff Preis"].to_dict())
model.nh3_limit = pyo.Param(model.country, initialize=input_data["NH3 Export Limit"].to_dict())

model.ch3oh_price =  pyo.Param(model.country, initialize=input_data["CH3OH Herstellungspreis"].to_dict())
model.ch3oh_pipeline =  pyo.Param(model.country, initialize=input_data["CH3OH Pipeline Preis"].to_dict())
model.ch3oh_ship =  pyo.Param(model.country, initialize=input_data["CH3OH Schiff Preis"].to_dict())
model.ch3oh_limit = pyo.Param(model.country, initialize=input_data["CH3OH Export Limit"].to_dict())

model.h2_price.pprint()

## Variablen

In [7]:
# Variablen definieren
model.h2_amount_ship = pyo.Var(model.country, domain = pyo.NonNegativeReals)
model.h2_amount_pipeline = pyo.Var(model.country, domain = pyo.NonNegativeReals)

model.nh3_amount_ship = pyo.Var(model.country, domain = pyo.NonNegativeReals)
model.nh3_amount_pipeline = pyo.Var(model.country, domain = pyo.NonNegativeReals)

model.ch3oh_amount_ship = pyo.Var(model.country, domain = pyo.NonNegativeReals)
model.ch3oh_amount_pipeline = pyo.Var(model.country, domain = pyo.NonNegativeReals)

## Zielfunktion

In [8]:
# Zielfunktion
obj_expr = sum(model.h2_pipeline[i]*model.h2_amount_pipeline[i] + model.h2_ship[i]*model.h2_amount_ship[i] + model.h2_price[i]*(model.h2_amount_pipeline[i] + model.h2_amount_ship[i]) +
               model.nh3_pipeline[i]*model.nh3_amount_pipeline[i] + model.nh3_ship[i]*model.nh3_amount_ship[i] + model.nh3_price[i]*(model.nh3_amount_pipeline[i] + model.nh3_amount_ship[i]) +
               model.ch3oh_pipeline[i]*model.ch3oh_amount_pipeline[i] + model.ch3oh_ship[i]*model.ch3oh_amount_ship[i] + model.ch3oh_price[i]*(model.ch3oh_amount_pipeline[i] + model.ch3oh_amount_ship[i]) for i in model.country)
model.obj = pyo.Objective(sense=pyo.minimize, expr=obj_expr)

# müssen noch gemacht werden

## Nebenbedingungen

In [9]:
# Bedarfsdeckung
model.demand_constraints = pyo.ConstraintList()

con_demand_h2_lhs = sum(model.h2_amount_pipeline[i] + model.h2_amount_ship[i] for i in model.country)
con_demand_h2_rhs = 10
model.demand_constraints.add(con_demand_h2_lhs >= con_demand_h2_rhs)

con_demand_nh3_lhs = sum(model.nh3_amount_pipeline[i] + model.nh3_amount_ship[i] for i in model.country)
con_demand_nh3_rhs = 10
model.demand_constraints.add(con_demand_nh3_lhs >= con_demand_nh3_rhs)

con_demand_ch3oh_lhs = sum(model.ch3oh_amount_pipeline[i] + model.ch3oh_amount_ship[i] for i in model.country)
con_demand_ch3oh_rhs = 10
model.demand_constraints.add(con_demand_ch3oh_lhs >= con_demand_ch3oh_rhs)

# Exportlimits
model.export_constraints = pyo.ConstraintList()

for i in model.country:
    con_export_h2_lhs = model.h2_amount_pipeline[i] + model.h2_amount_ship[i]
    con_export_h2_rhs = model.h2_limit[i]
    model.export_constraints.add(con_export_h2_lhs <= con_export_h2_rhs)
    
    con_export_nh3_lhs = model.nh3_amount_pipeline[i] + model.nh3_amount_ship[i]
    con_export_nh3_rhs = model.nh3_limit[i]
    model.export_constraints.add(con_export_nh3_lhs <= con_export_nh3_rhs)
    
    con_export_ch3oh_lhs = model.ch3oh_amount_pipeline[i] + model.ch3oh_amount_ship[i]
    con_export_ch3oh_rhs = model.ch3oh_limit[i]
    model.export_constraints.add(con_export_ch3oh_lhs <= con_export_h2_rhs)


model.pprint()

In [10]:
# Create a model instance and optimize
instance = model.create_instance()
results = solver.solve(instance)
instance.display()

Model unknown

  Variables:
    h2_amount_ship : Size=4, Index=h2_amount_ship_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :   0.0 :  None : False : False : NonNegativeReals
          B :     0 :   0.0 :  None : False : False : NonNegativeReals
          C :     0 :   0.0 :  None : False : False : NonNegativeReals
         De :     0 :   0.0 :  None : False : False : NonNegativeReals
    h2_amount_pipeline : Size=4, Index=h2_amount_pipeline_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :   3.0 :  None : False : False : NonNegativeReals
          B :     0 :   4.0 :  None : False : False : NonNegativeReals
          C :     0 :   3.0 :  None : False : False : NonNegativeReals
         De :     0 :   0.0 :  None : False : False : NonNegativeReals
    nh3_amount_ship : Size=4, Index=nh3_amount_ship_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :   0.0 :  None : False